<a href="https://colab.research.google.com/github/ahmadhajmosa/Machine-learning-labs/blob/master/Session_3_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab on Machine Learning and Applications in Intelligent Vehicles
## Session 3: Keras

### * Intro into Keras

### * CNN using Keras

### * LSTM using Keras

### * Variational Auto encoder using Keras

### * Seq2Seq Model with keras


In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(32, input_shape=(16,),name="input_layer"))
model.add(Dense(100, activation="relu",name="hidden_layer_1"))
model.add(Dropout(0.1))

model.add(Dense(200, activation="sigmoid",name="hidden_layer_2"))
model.add(Dropout(0.1))
model.add(Dense(30,name="hidden_layer_3"))
model.add(Dense(100,name="hidden_layer_4"))

# now the model will take as input array
#

#
#
model.add(Dense(32,name="output_layer"))

#
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot',format='svg'))

display(model.summary()) # if this is left out, you get a diagram (Note by Mareike)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 32)                544       
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 100)               3300      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
hidden_layer_2 (Dense)       (None, 200)               20200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 200)               0         
_________________________________________________________________
hidden_layer_3 (Dense)       (None, 30)                6030      
_________________________________________________________________
hidden_layer_4 (Dense)       (None, 100)               3100      
__________

None

In [14]:
from keras.layers import Conv2D, Flatten

model = Sequential()
model.add(Conv2D(64, (3,3), 
                 input_shape=(3,32,32), padding='same',))
model.add(Conv2D(20, (3,3), padding='same'))

#now: model.output_shape == (None, 64, 32, 32)
#now: model.input_shape == (None, 3, 32, 32)

model.add(Flatten())

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot',format='svg'))

display(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 32, 64)         18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 32, 20)         11540     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
Total params: 30,036
Trainable params: 30,036
Non-trainable params: 0
_________________________________________________________________


None